In [1]:
import os
exp_data = '{"name": "moremetrices01", "algo_details": {"snowflake.ml.modeling.ensemble.GradientBoostingClassifier": null}, "id": "367", "dataset": "AIRLINE_DEP_DELAY_10K", "target_column": "DEP_DEL15"}'

In [2]:
import logging, sys, os
from snowflake.snowpark.session import Session

In [3]:
CONNECTION_PARAMETERS = {
    "account": "ug94937.us-east4.gcp",
    "user":"ADITYASINGH",
    "password": os.environ.get('SF_Password'),
    "role": "ADITYASINGH",
    "database": "FIRST_DB",
    "warehouse": "FOSFOR_INSIGHT_WH",
    "schema": "PUBLIC"
}


In [24]:
def create_stage(session, stage_name="demo"):
    try:
        session.sql(f"create or replace stage {stage_name}").collect()
        return f"@{stage_name}"
    except Exception as ex:
        print("Error while creating snowflake session", ex)
        raise ex

def get_session():
    """
    Method creates snowflake session object.
    :return:
    """
    try:
        return Session.builder.configs(CONNECTION_PARAMETERS).create()
    except Exception as ex:
        print("Error while creating snowflake session", ex)
        raise ex


# Stored Procedure
def train_ml_models(session: Session, exp_data: str) -> list:
    # variable for holding logs
    logs = []
    
    # function for accumulating logs
    def log_message(level: str, message: str):
        logs.append(f"{level}: {message}")
        
    from snowflake.ml.modeling.pipeline import Pipeline
    from snowflake.ml.modeling.preprocessing import MinMaxScaler, LabelEncoder, OneHotEncoder
    from snowflake.ml.modeling.metrics import mean_squared_error, mean_absolute_error, \
         r2_score, accuracy_score, precision_score, roc_auc_score, f1_score, recall_score
    from snowflake.snowpark.functions import col, last_value, row_number, first_value, regexp_replace
    from snowflake.snowpark.window import Window
    from snowflake.snowpark.types import StringType
    import importlib, sys, json


    log_message("INFO","Starting Experiment Recipe Execution")
    
    # Experiment details
    exp_details=json.loads(exp_data)
    
    # Read dataset, Random split
    log_message("INFO","Reading and Identifing dataset features")
    data = session.table(exp_details.get("dataset"))
    
    # Data Preprocessing: Validating and encoding the data if required and imputing null values.
    window_spec = Window.order_by(exp_details.get("target_column"))
    data_row_num = data.with_column('ROW_NUM', row_number().over(window_spec))
    columns = data_row_num.columns
    columns.remove('ROW_NUM')
    # forward fillna
    data_ff = data_row_num.select(
            [data_row_num['ROW_NUM']] + [
                last_value(col(column), ignore_nulls=True).over(Window.order_by('ROW_NUM')).alias(column)
                for column in columns
            ]
        )
    # backward fillna
    data_bf = data_row_num.select(
            [data_row_num['ROW_NUM']] + [
                first_value(col(column), ignore_nulls=True).over(Window.order_by(data_ff['ROW_NUM'].desc())).alias(column)
                for column in columns
            ]
        )
    data = data_bf.drop('ROW_NUM')
    
    # Replace special character(- with _)
    schema_fields = data.schema.fields
    data = data.select([
        regexp_replace(regexp_replace(col(field.name),"-", "_"), "\\").alias(field.name) if isinstance(field.datatype, StringType) else col(field.name)
        for field in schema_fields])
    print(schema_fields)
    data.show()
    schema_fields = data.schema.fields
    print("schhema after special char handling : ",schema_fields)
    features = data.columns
#     features.remove(exp_details.get("target_column"))
    
    # get features
    data_schema = session.sql(f"DESCRIBE TABLE {exp_details.get('dataset')}").collect()
    categorical_types = ['VARCHAR','CHAR','STRING','TEXT','BOOL']
    categorical_features = []  
    for row in data_schema:
        for typ in categorical_types:
            if typ in row['type']:
                categorical_features.append(row['name'])
                break
    numerical_features = list(set(features) - set(categorical_features))
    print(f"numerical_features:  {numerical_features}")
    print(f"categorical_features_oe: {categorical_features}")
    log_message("INFO",f"numerical_features:  {numerical_features}")
    log_message("INFO",f"categorical_features_oe: {categorical_features}")
    
    #identify columns for labelencoding and onehotencoding   
    le_column_feature = []
    oh_column_feature = []
    if len(categorical_features) >= 1:
        print(f"{categorical_features} columns are non numeric in feature dataset, encoding required.")
        for column in categorical_features:
            if data.select(data[column]).distinct().count() >= 10:
                le_column_feature.append(column)
            else:
                oh_column_feature.append(column)

        log_message("INFO",f"Columns identified to be encoded with label encoder: {le_column_feature}")
        log_message("INFO",f"Columns identified to be encoded with one hot encoder: {oh_column_feature}")
    
    
    #pipeline steps 
    log_message("INFO","Setting up preprocessing pipeline based on dataset")
    categorical_pp = {
        'label_enc': [LabelEncoder(input_cols=le_column_feature, output_cols=le_column_feature), le_column_feature],
        'oh_enc': [OneHotEncoder(input_cols=oh_column_feature, output_cols=oh_column_feature), oh_column_feature]
    }
    numerical_pp = {
        'scaler': MinMaxScaler(input_cols=numerical_features, output_cols=numerical_features)
    }
    steps = [(key, categorical_pp[key][0]) for key in categorical_pp if categorical_pp[key][1]!=[]] + \
    [(key, numerical_pp[key]) for key in numerical_pp if numerical_features!=[]]
    
    
    input_cols = categorical_features+numerical_features
    input_cols.remove(exp_details.get("target_column"))
    df_train, df_test = data.drop('ROW').random_split(weights=[0.9, 0.1], seed=0)
    
    # Define a pipeline that does the preprocessing and training of 
    # dynamically import selected algorithms
    for algorithm, hyperparam in exp_details.get("algo_details").items():
        algorithm = algorithm.rsplit('.', 1)
        module = importlib.import_module(algorithm[0])
        log_message("INFO",f"Running Algorithm {algorithm[1]}")
        attr = getattr(module, algorithm[1])
        
        pipe = Pipeline(steps=steps+[("algorithm", attr(input_cols=input_cols
                                              , label_cols=[exp_details.get("target_column")]
                                              , output_cols=[f'PREDICTIONS_{algorithm[1]}'.upper()]))]
               )

        # Fit the pipeline
        log_message("INFO",f"Running model pipeline {algorithm[1]}")
        model = pipe.fit(df_train)
 
        # Test the model
        log_message("INFO","Running prediction on model with test dataset")
        df_test_pred = model.predict(df_test)
        
        # metrices
        log_message("INFO","Generating Metrices")
        accuracy = accuracy_score(df=df_test_pred, y_true_col_names=exp_details.get("target_column"), y_pred_col_names=f'PREDICTIONS_{algorithm[1]}'.upper())
        f1_score = f1_score(df=df_test_pred, y_true_col_names=exp_details.get("target_column"), y_pred_col_names=f'PREDICTIONS_{algorithm[1]}'.upper())
        recall_score = recall_score(df=df_test_pred, y_true_col_names=exp_details.get("target_column"), y_pred_col_names=f'PREDICTIONS_{algorithm[1]}'.upper())
        precision_score = precision_score(df=df_test_pred, y_true_col_names=exp_details.get("target_column"), y_pred_col_names=f'PREDICTIONS_{algorithm[1]}'.upper())
        roc_auc_score = roc_auc_score(df=df_test_pred, y_true_col_names=exp_details.get("target_column"), y_score_col_names=f'PREDICTIONS_{algorithm[1]}'.upper())
        print("Execution Completed")
        print(f'{algorithm[1]} MSE: {accuracy}')
        print(f'{algorithm[1]} R2: {precision_score}')
        print(f'{algorithm[1]} R2: {roc_auc_score}')
        

        # LOG MODEL INTO SNOWFLAKE REGISTRY
        from snowflake.ml.registry.registry import Registry
        reg = Registry(session=session)
        # Log the model
        log_message("INFO","Started: Registering model on snowflake")
        try:
            mv = reg.log_model(model=model,
                               model_name=exp_details.get("name", "sample_experiment")+"_"+algorithm[1],
                               comment="test",
                               version_name="run1",
                               python_version="3.9.19",
                               conda_dependencies=["scikit-learn==1.3.2"],
                               metrics=[{"model_metrics": {"roc_auc_score": roc_auc_score, "precision_score": precision_score, "f1_score": f1_score, "recall_score": recall_score, "accuracy_score": accuracy}, "project_id": "0001", "type": "EXP"}])
            log_message("INFO","Registeration of model completed!!!")
        except Exception as ex:
            key = 'Processing aborted due to error 370001' 
            if key in str(ex):
                log_message("INFO","Registeration of model completed!!!")
                pass
            else:
                log_message("ERROR","Exception Occured while registering model")
                return str(ex).split('?')
    return [{"Execution Logs:": "\n".join(logs),
             "EXP_NAME":exp_details.get("name", "sample_experiment"),
             "Version":"Run1",
             "matrices":{"model_metrics": {"roc_auc_score": roc_auc_score, "precision_score": precision_score, "f1_score": f1_score, "recall_score": recall_score, "accuracy_score": accuracy}, "project_id": "0001", "type": "EXP"},
             "Alogirthm_Type":"Regression",
             "Alogithms": list(exp_details.get("algo_details").keys()),
             "RUN_STATUS":"SUCCESS",
             "registry_exp_name":""}]

In [25]:
# Initilization
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
print("Creating Snowflake Session object...")
session = get_session()
stage = create_stage(session)
print("Session has been created !")

print("Creating stored procedure...")
session.sproc.register(func=train_ml_models,
                       name="train_ml_models",
                       packages=["snowflake-snowpark-python", "snowflake-ml-python"],
                       isPermanant=False,
                       stage_location=stage,
                       replace=True)
print("Stored procedure has been created successfully!")

print("Executing Procedure")
# procedure_response = session.call("train_ml_models", exp_data)
procedure_response = train_ml_models(session, exp_data)
print("Stored Procedure Executed Successfully !")
print(procedure_response)

#Log in mlflow
print("Logging in mlflow completed !")

Creating Snowflake Session object...
INFO:snowflake.connector.connection:Snowflake Connector for Python Version: 3.10.1, Python Version: 3.9.18, Platform: Linux-5.10.215-203.850.amzn2.x86_64-x86_64-with-glibc2.34
INFO:snowflake.connector.connection:This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.
INFO:snowflake.snowpark.session:Snowpark Session information: 
"version" : 1.18.0,
"python.version" : 3.9.18,
"python.connector.version" : 3.10.1,
"python.connector.session.id" : 96125691344382,
"os.name" : Linux

INFO:snowflake.connector.cursor:Number of results in first chunk: 1
Session has been created !
Creating stored procedure...
INFO:snowflake.connector.cursor:Number of results in first chunk: 0
INFO:snowflake.connector.cursor:Number of results in first chunk: 2
INFO:snowflake.connector.cursor:Number

schhema after special char handling :  [StructField('MONTH', LongType(), nullable=True), StructField('DAY_OF_WEEK', LongType(), nullable=True), StructField('DEP_DEL15', LongType(), nullable=True), StructField('DEP_TIME_BLK', StringType(16777216), nullable=True), StructField('DISTANCE_GROUP', LongType(), nullable=True), StructField('SEGMENT_NUMBER', LongType(), nullable=True), StructField('CONCURRENT_FLIGHTS', LongType(), nullable=True), StructField('NUMBER_OF_SEATS', LongType(), nullable=True), StructField('CARRIER_NAME', StringType(16777216), nullable=True), StructField('AIRPORT_FLIGHTS_MONTH', LongType(), nullable=True), StructField('AIRLINE_FLIGHTS_MONTH', LongType(), nullable=True), StructField('AIRLINE_AIRPORT_FLIGHTS_MONTH', LongType(), nullable=True), StructField('AVG_MONTHLY_PASS_AIRPORT', LongType(), nullable=True), StructField('AVG_MONTHLY_PASS_AIRLINE', LongType(), nullable=True), StructField('FLT_ATTENDANTS_PER_PASS', DoubleType(), nullable=True), StructField('GROUND_SERV_P

/packages/Python-3.9-Snowpark/7e0b365a-f7cb-4d2a-ba2a-e8ed02dbe50c/3.9/snowflake/ml/modeling/preprocessing/one_hot_encoder.py:845: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  state_pandas[[_CATEGORY]] = state_pandas[[_CATEGORY]].applymap(convert_to_string_excluding_nan)
/packages/Python-3.9-Snowpark/7e0b365a-f7cb-4d2a-ba2a-e8ed02dbe50c/3.9/snowflake/ml/modeling/preprocessing/one_hot_encoder.py:846: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  state_pandas[[_FITTED_CATEGORY]] = state_pandas[[_FITTED_CATEGORY]].applymap(convert_to_string_excluding_nan)


INFO:snowflake.connector.cursor:Number of results in first chunk: 1
INFO:snowflake.connector.cursor:Number of results in first chunk: 11
INFO:snowflake.connector.cursor:Number of results in first chunk: 1
INFO:snowflake.connector.cursor:Number of results in first chunk: 1
INFO:snowflake.connector.cursor:Number of results in first chunk: 0
INFO:snowflake.connector.cursor:Number of results in first chunk: 0
INFO:snowflake.connector.cursor:Number of results in first chunk: 0
INFO:snowflake.connector.cursor:Number of results in first chunk: 0
INFO:snowflake.connector.cursor:Number of results in first chunk: 0
INFO:snowflake.connector.cursor:Number of results in first chunk: 0
INFO:snowflake.connector.cursor:Number of results in first chunk: 0
INFO:snowflake.connector.cursor:Number of results in first chunk: 0
INFO:snowflake.connector.cursor:Number of results in first chunk: 0
INFO:snowflake.connector.cursor:Number of results in first chunk: 0
INFO:snowflake.connector.cursor:Number of resul

/packages/Python-3.9-Snowpark/7e0b365a-f7cb-4d2a-ba2a-e8ed02dbe50c/3.9/snowflake/ml/modeling/preprocessing/one_hot_encoder.py:845: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  state_pandas[[_CATEGORY]] = state_pandas[[_CATEGORY]].applymap(convert_to_string_excluding_nan)
/packages/Python-3.9-Snowpark/7e0b365a-f7cb-4d2a-ba2a-e8ed02dbe50c/3.9/snowflake/ml/modeling/preprocessing/one_hot_encoder.py:846: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  state_pandas[[_FITTED_CATEGORY]] = state_pandas[[_FITTED_CATEGORY]].applymap(convert_to_string_excluding_nan)


INFO:snowflake.connector.cursor:Number of results in first chunk: 1
INFO:snowflake.connector.cursor:Number of results in first chunk: 11
INFO:snowflake.connector.cursor:Number of results in first chunk: 1
INFO:snowflake.connector.cursor:Number of results in first chunk: 1
INFO:snowflake.connector.cursor:Number of results in first chunk: 0
INFO:snowflake.connector.cursor:Number of results in first chunk: 0
INFO:snowflake.connector.cursor:Number of results in first chunk: 0
INFO:snowflake.connector.cursor:Number of results in first chunk: 0
INFO:snowflake.connector.cursor:Number of results in first chunk: 0
INFO:snowflake.connector.cursor:Number of results in first chunk: 0
INFO:snowflake.connector.cursor:Number of results in first chunk: 0
INFO:snowflake.connector.cursor:Number of results in first chunk: 0
INFO:snowflake.connector.cursor:Number of results in first chunk: 0
INFO:snowflake.connector.cursor:Number of results in first chunk: 0
INFO:snowflake.connector.cursor:Number of resul

SnowparkSQLException: (1300) (1304): 01b4f671-0000-68a2-0000-576d06e6affe: 100357 (P0000): 01b4f671-0000-68a2-0000-576d06e6affe: Python Interpreter Error:
Traceback (most recent call last):
  File "_udf_code.py", line 77, in compute
  File "/packages/Python-3.9-Snowpark/7e0b365a-f7cb-4d2a-ba2a-e8ed02dbe50c/3.9/snowflake/ml/modeling/_internal/snowpark_implementations/snowpark_trainer.py", line 234, in fit_wrapper_function
  File "/usr/lib/python_udf/46488fdc5979524e9584c22464c1120c408c5be892bf98662b3505d23b94117a/lib/python3.9/site-packages/sklearn/base.py", line 1151, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/usr/lib/python_udf/46488fdc5979524e9584c22464c1120c408c5be892bf98662b3505d23b94117a/lib/python3.9/site-packages/sklearn/ensemble/_gb.py", line 416, in fit
    X, y = self._validate_data(
  File "/usr/lib/python_udf/46488fdc5979524e9584c22464c1120c408c5be892bf98662b3505d23b94117a/lib/python3.9/site-packages/sklearn/base.py", line 621, in _validate_data
 